In [3]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from neural_network import NeuralNetwork

In [5]:
sizes = [3,5,4,2]

network = NeuralNetwork(sizes)
print(network._weights)

[array([[-0.14115403, -0.01358996,  0.56304016,  0.06606496],
       [-0.41679373, -0.27652526, -0.20647908, -0.19082555],
       [-0.46780175, -0.28421479, -0.2543726 ,  0.49051808],
       [ 0.00967069, -0.52467735, -0.54378305, -0.15178162],
       [ 0.22729764, -0.53137237, -0.18534303,  0.40126338]]), array([[ 0.11052946, -0.15659258,  0.39846295,  0.23311869, -0.1385221 ,
        -0.29236302],
       [ 0.21467536, -0.07237006,  0.25403479, -0.11290637, -0.03378731,
        -0.34808663],
       [ 0.3611357 ,  0.06330553,  0.0894542 ,  0.24630316, -0.29182526,
         0.27275657],
       [ 0.16550032, -0.1834235 ,  0.31570629,  0.37923761,  0.03581473,
        -0.27610299]]), array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])]
